# ルートを評価するために解くLPについて
- ルートが1つ与えられた後、そのルートを評価するためにLPを解く(ルートが実行可能であるとは限らない)

<!--- 各制約の違反度を変数とし、それらに重みをかけて足し合わせた関数の最小化問題とする-->
- 車両が各顧客へ到着する時刻を変数とし、その合計を最小化する問題とする
- 制約は、容量制約と時間枠制約とする
    - 容量制約は、ある区間における2つの関数の積分値(面積)の大小を比較するという形で表す(車両の積荷の量を表す区分線形関数の\[x0, xn\]までの積分値と最大容量を表す線形関数の\[x0, xn\]までの積分値)
    - 時間枠制約は、通常のVRPの定式化と同様に表す


# 前準備

## instances.pyからインスタンスを得る

In [1]:
import instances
Customers = instances.Customers
tour = instances.tour

In [2]:
instance_name = "C" + str(len(Customers)-1)

In [3]:
tour[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
tour[-10:]

[50, 51, 52, 53, 54, 55, 56, 57, 58, 'depot']

# 問題を解く
2つの手法に対し、計算時間を比較

## 入力する行列、ベクトルの作成

In [5]:
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

In [6]:
def make_preceding_constr(tour, Customers, Ax, Ap, b, c):
    # ルート内の顧客の順序に関する制約
    for index, i in enumerate(tour):
        try:
            i_next = tour[index+1]
        except:
            i_next = "depot"
            continue
        Ax.append([1 if target==i else -1 if target==i_next else 0 for target in tour])
        Ap.append([0 for _ in range(len(tour))])
        b.append(-distance(Customers[i].x, Customers[i].y, Customers[i_next].x, Customers[i_next].y))
    return Ax, Ap, b, c

In [7]:
def make_tw_constr(tour, Customers, Ax, Ap, b, c):
    # 時間枠制約
    for index, i in enumerate(tour):
        Ax.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(-Customers[i].e)
        Ax.append([1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b.append(Customers[i].l)
    return Ax, Ap, b, c

In [8]:
def make_inputs(tour, Customers):
    import numpy as np
    Ax, Ap, b, c = [], [], [], [1 for _ in range(len(tour))]
    # ルート内の顧客の順序に関する制約
    Ax, Ap, b, c = make_preceding_constr(tour, Customers, Ax, Ap, b, c)
    print("Preceding constraints are done.")
    # 時間枠制約
    Ax, Ap, b, c = make_tw_constr(tour, Customers, Ax, Ap, b, c)
    print("Time-window constarints are done.")
    # ndarrayに変換
    Ax = np.array(Ax)
    Ap = np.array(Ap)
    b = np.array(b)
    c = np.array(c)
    return Ax, Ap, b, c

## 主問題を解く関数の定義
与えられた定数を元にLPのモデルを作成した上でそれを解き、最適解を返す関数

In [9]:
def solve_primal(Ax, Ap, b, c, instance_name, num_vars):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    # 定数を設定　←　入力として与えられる
    # 変数を設定
    """
    x_i : 顧客iへ車両が到着する時刻を表す変数
    p_i : 顧客iの時間枠の違反度を表す変数
    """
    x = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="x")
    p = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="p")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(c.T @ p, sense=gp.GRB.MINIMIZE)
    
    # 制約条件を設定
    m.addConstr(Ax @ x + Ap @ p <= b, name="c")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    """if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"車両が顧客{i}に到着する時刻は、{x[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')"""
    
    # モデルをテキストファイルにする
    m.write("out"+instance_name+".json")
        
    return m

## 双対問題を解く関数の定義

In [24]:
def solve_dual(Ax, Ap, b, c, instance_name, num_vars, PStarts, DStarts):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    
    # 変数を設定
    """
    y_i : 主問題における制約式iの潜在価値
    """
    y = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="y")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(-1 * b.T @ y, sense=gp.GRB.MAXIMIZE)
    
    # 制約条件を設定
    c1 = m.addConstr(Ax.T @ y >= 0, name="c1")
    c2 = m.addConstr(Ap.T @ y + c >= 0, name="c2")
    c3 = m.addConstr(y >= 0, name="c3")
    c = c1+ c2 + c3

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # ホットスタートの使用
    for i in range(num_vars):
        y[i].PStart = PStarts[i]
    for i in range(len(c)):
        c[i].DStart = DStarts[i]
    
    # 最適化
    m.Params.Method = 0
    #m.Params.Presolve = 0
    
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    """if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"主問題における制約{i}の潜在価格は、{y[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')"""
    
    # モデルをテキストファイルにする
    m.write("out"+instance_name+".mst")
        
    return m

## ①全体を1つのLPとして解くveb.

### 全体のPrimalを解く

In [11]:
Ax, Ap, b, c = make_inputs(tour, Customers)

Preceding constraints are done.
Time-window constarints are done.


In [12]:
# Gurobiによって最適解を求める
P = solve_primal(Ax, Ap, b, c, instance_name+"P", len(tour))

Using license file /Users/okamoto/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 179 rows, 120 columns and 358 nonzeros
Model fingerprint: 0xeedfc6f2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 167 rows and 110 columns
Presolve time: 0.07s
Presolved: 12 rows, 10 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0418432e+03   2.162926e+00   0.000000e+00      0s
       7    4.0445317e+03   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.09 seconds
Optimal objective  4.044531741e+03


In [ ]:
for var in P.getVars():
    print(var.varName, var.X)

In [ ]:
for constr in P.getConstrs():
    print(constr.Pi)

## ②前半と後半をつなげるveb.
1. 適当なところで前後に分ける
1. 前半後半それぞれのPrimalを解く
1. 前半と後半それぞれのPrimalの最適解を、全体のDualに入れて解く
1. 全体のPrimalの最適解を得る

### 1. 適当なところで前後に分ける
- ひとまず半分くらいで分けることにする

In [13]:
threshold = len(tour)//2
#print(threshold)

former = tour[:threshold]
latter = tour[threshold:]
#print(f"巡回路全体は、{tour}")
print(f"前半は、{former}")
print(f"後半は、{latter}")

前半は、[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
後半は、[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 'depot']


### 2. 前半後半それぞれのPrimalを解く

In [14]:
import numpy as np
# 係数行列、ベクトルを整える
A1x, A1p, A2x, A2p, A3x, A3p, b1, b2, b3, c1, c2 = [], [], [], [], [], [], [], [], [], [], []
index = 0
index_P_f, index_P_l = [], []
## A1, A2, A3を定める
## b1, b2, b3を定める
for Ax_i, Ap_i, b_i in zip(Ax, Ap, b):
    if np.linalg.norm(Ax_i[threshold:], ord=2)==0.:
        A1x.append(Ax_i[:threshold])
        A1p.append(Ap_i[:threshold])
        b1.append(b_i)
        index_P_f.append(index)
    elif np.linalg.norm(Ax_i[:threshold], ord=2)==0.:
        A2x.append(Ax_i[threshold:])
        A2p.append(Ap_i[threshold:])
        b2.append(b_i)
        index_P_l.append(index)
    else:
        A3x.append(Ax_i)
        A3p.append(Ap_i)
        b3.append(b_i)
    index += 1
## c1, c2, c3を定める
c1 = c[:threshold]
c2 = c[threshold:]
## リストからnumpy arrayに変換
A1x = np.array(A1x)
A1p = np.array(A1p)
A2x = np.array(A2x)
A2p = np.array(A2p)
A3x = np.array(A3x)
A3p = np.array(A3p)
b1 = np.array(b1)
b2 = np.array(b2)
b3 = np.array(b3)
c1 = np.array(c1)
c2 = np.array(c2)
for i in range(1, 4):
    print(f"A{i}x : ", end="")
    print(eval("A"+str(i)+"x"))
    print(f"A{i}p : ", end="")
    print(eval("A"+str(i)+"p"))
    print(f"b{i} : ", end="")
    print(eval("b"+str(i)))
    if i <= 2:
        print(eval("c"+str(i)))

A1x : [[ 1 -1  0 ...  0  0  0]
 [ 0  1 -1 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0  1]]
A1p : [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  0  0 -1]]
b1 : [  -6.08276253   -3.60555128   -5.83095189   -7.           -2.
  -11.3137085   -11.40175425   -8.54400375   -3.60555128   -8.06225775
   -6.70820393  -10.81665383   -5.83095189   -4.           -2.23606798
   -2.23606798   -6.           -3.           -4.47213595   -1.41421356
   -2.           -6.32455532   -4.12310563   -7.28010989   -7.81024968
  -10.81665383   -8.06225775   -7.81024968   -8.48528137   -2.
    6.           -7.           10.          -10.           14.
  -14.           14.          -16.           21.          -22.
   22.          -23.           26.          -27.           30.
  -32.           33.          -34.           38.          -40.
   40.   

In [15]:
# Gurobiによって最適解を求める
P_f = solve_primal(A1x, A1p, b1, c1, instance_name+"P_f", len(former))
P_l = solve_primal(A2x, A2p, b2, c2, instance_name+"P_l", len(latter))

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 89 rows, 60 columns and 178 nonzeros
Model fingerprint: 0x72456f1b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 78 rows and 51 columns
Presolve time: 0.04s
Presolved: 11 rows, 9 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0767780e+02   2.162926e+00   0.000000e+00      0s
       7    8.1016095e+02   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.05 seconds
Optimal objective  8.101609507e+02
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 89 rows, 60 columns and 178 nonzeros
Model fingerprint: 0x4b34acdd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2 

In [ ]:
for v in P_f.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for v in P_l.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for constr in P_f.getConstrs()+P_l.getConstrs():
    print(constr.Pi, constr.CBasis)

### 3. 前半後半それぞれのPrimalの最適解を、全体のDualに入れて解く

In [16]:
# 初期解の保存
PStarts = np.array([constr.Pi for constr in P_f.getConstrs()+P_l.getConstrs()])
y3 = np.zeros((Ax.shape[0]-PStarts.shape[0],))
PStarts = np.append(PStarts, y3)

DStarts = np.array([0 for var in P_f.getVars()+P_l.getVars()])
c3 = np.zeros((PStarts.shape[0],))
DStarts = np.append(DStarts, c3)
for i, var in zip(index_P_f, P_f.getVars()):
    DStarts[i] = var.x
for i, var in zip(index_P_l, P_l.getVars()):
    DStarts[i] = var.x

In [ ]:
print(f"Ax.T.shape={Ax.T.shape}\t\t\tAp.T.shape={Ap.T.shape}")
for Ax_i, Ap_i in zip(Ax.T, Ap.T):
    print(Ax_i, "\t", Ap_i)

In [20]:
PStarts = np.array([constr.Pi for constr in P.getConstrs()])
DStarts = np.array([var.x for var in P.getVars()])
c3 = np.zeros((PStarts.shape[0],))
DStarts = np.append(DStarts, c3)

In [25]:
# Gurobiによって最適解を求める
D = solve_dual(Ax, Ap, b, c, instance_name+"D", PStarts.shape[0], PStarts, DStarts)

Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 299 rows, 179 columns and 537 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Crossover log...

     118 DPushes remaining with DInf 2.3772580e+04                 0s
       0 DPushes remaining with DInf 4.5546330e+03                 0s

       0 PPushes remaining with PInf 0.0000000e+00                 0s

  Push phase complete: Pinf 0.0000000e+00, Dinf 4.5546330e+03      0s

Iteration    Objective       Primal Inf.    Dual Inf.      Time
      61   -0.0000000e+00   0.000000e+00   4.554633e+03      0s

Solved in 122 iterations and 0.06 seconds
Optimal objective  4.044531741e+03


In [ ]:
# 最適解
for var in D.getVars():
    print(var.varName, var.X)

### 4. 全体のPrimalの最適解を得る

In [ ]:
for i, constr in enumerate(D.getConstrs()):
    print(constr.Pi)

In [ ]:
for var in P.getVars():
    print(var.varName, var.X)